In [1]:
import boto3
import spotipy
import json
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np

In [ ]:
# TODO
# Artist seed needs to search song first and get value that best matches target value 
# Normalize values

In [68]:
client = boto3.client('lambda')
response = client.invoke(FunctionName='spotifyLambda', InvocationType='RequestResponse')
data = json.loads(response['Payload'].read())
token = json.loads(data['body'])['token']

In [69]:
sp = spotipy.Spotify(token)

In [4]:
def getFeatures(songID):
    audioFeaturesRaw = sp.audio_features(songID)[0]    
    return audioFeaturesRaw

In [93]:
def getRecommendations(ids, target_danceability):
#     print("Recommendations: ", ids)
    recommendations = sp.recommendations(seed_tracks=ids, target_danceability=target_danceability)
    
    return recommendations['tracks']

In [79]:
def generatePlaylist(numTracks, targets, seeds):
    
    """
    numTracks: number of tracks in the set
    intervals: the target danceability
    seeds: dictionary of spotify id of artist or song, and location
    """
        
    playlist = np.empty(shape=numTracks, dtype=np.object)
    
    for seed in seeds:
        seedIndex = seed['location']
        playlist[seedIndex] = seed['id']
        
#         print(seed)
#         rec = getRecommendations([seed['id']], 0.9)
#         print(rec[0])
#         playlist[seed['location']] = rec[0]['id']
        
    for i in range(seedIndex+1, len(playlist)):
        num_seeds = min(i-seedIndex, 5)
        trackseeds = playlist[seedIndex:seedIndex+num_seeds]
        recommendations = getRecommendations(list(trackseeds), 0.9)
        for track in recommendations:
            if track['id'] not in playlist:
                playlist[i] = track['id']

    print(playlist)

    for i in range(seedIndex-1, -1, -1):
        x = len(playlist)-i
        print(x)
        if x >= 5:
            x = 5
        trackseeds = playlist[i+1:i+x+1]
        recommendations = getRecommendations(list(trackseeds), 0.9)
        for track in recommendations:
            if track['id'] not in playlist:
                playlist[i] = track['id']
        
        
    return playlist  
#     for i in range(seedIndex-1, -1, -1):
#         num_seeds = min(i-seedIndex, (len(playlist)-seedIndex), 5)
    
#     for i in range(seedIndex, 0, -1):
        
#     # 1. Get first song using song seed & get danceability

#     print(playlist)
    
    
    
    # 2. For forward and backwards:
    #    target = diff(previous value*intervals[i])
    #    get recommended songs with target
    #    iterate over results to ensure no duplicates

In [80]:
seeds = [{
    'location': 6,
    'id': 'spotify:track:22bNlvnDQbYAq0Ex2sduSu'
}]
playlist = generatePlaylist(11, None, seeds)

Recommendations:  ['spotify:track:22bNlvnDQbYAq0Ex2sduSu']
[None None None None None None None None None
 'spotify:track:22bNlvnDQbYAq0Ex2sduSu' '442p4aC62fO5g7dBdVEzd0']
3
Recommendations:  ['spotify:track:22bNlvnDQbYAq0Ex2sduSu', '442p4aC62fO5g7dBdVEzd0']
4
Recommendations:  ['39DGA6wVmeA4cSZqD4UDqm', 'spotify:track:22bNlvnDQbYAq0Ex2sduSu', '442p4aC62fO5g7dBdVEzd0']
5
Recommendations:  ['2IA4VDpFh0pyY2Pb3CYKHf', '39DGA6wVmeA4cSZqD4UDqm', 'spotify:track:22bNlvnDQbYAq0Ex2sduSu', '442p4aC62fO5g7dBdVEzd0']
6
Recommendations:  ['3xwoix3bonSRudOF5VKJH6', '2IA4VDpFh0pyY2Pb3CYKHf', '39DGA6wVmeA4cSZqD4UDqm', 'spotify:track:22bNlvnDQbYAq0Ex2sduSu', '442p4aC62fO5g7dBdVEzd0']
7
Recommendations:  ['2jfO4JD0df6lXdmTRcUAVL', '3xwoix3bonSRudOF5VKJH6', '2IA4VDpFh0pyY2Pb3CYKHf', '39DGA6wVmeA4cSZqD4UDqm', 'spotify:track:22bNlvnDQbYAq0Ex2sduSu']
8
Recommendations:  ['29kEoIucvGtsu0MqXqN2PX', '2jfO4JD0df6lXdmTRcUAVL', '3xwoix3bonSRudOF5VKJH6', '2IA4VDpFh0pyY2Pb3CYKHf', '39DGA6wVmeA4cSZqD4UDqm']
9
Recomme

In [85]:
sp.user_playlist_create(name="Test Playlist", user="jup118")

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/12TDVXTouMN8fv5AuE27yg'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/12TDVXTouMN8fv5AuE27yg',
 'id': '12TDVXTouMN8fv5AuE27yg',
 'images': [],
 'name': 'Test run',
 'owner': {'display_name': 'jup118',
  'external_urls': {'spotify': 'https://open.spotify.com/user/jup118'},
  'href': 'https://api.spotify.com/v1/users/jup118',
  'id': 'jup118',
  'type': 'user',
  'uri': 'spotify:user:jup118'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSxhNTViNWU2NjBmZjdjZTFjNzY0YTc4YWZiZmU3MmU5YTI5MjMxMTQ1',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/12TDVXTouMN8fv5AuE27yg/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:12TDVXTouMN8fv5AuE27yg'}

In [92]:
sp.playlist_add_items(items=playlist, playlist_id="12TDVXTouMN8fv5AuE27yg")

{'snapshot_id': 'MyxhMmVkMzkwZmYzNDgwMzhkMjQwZmEyMDBhZTk5MDhmOGVkZmNkNWJm'}